In [1]:
import ipynb.fs.full.provenance as pr
import ipynb.fs.full.provenance_lib as pr_lib
import pandas as pd
import numpy as np
import time

from IPython.display import Image

# Specify where to save the processed files as savepath
savepath = 'results/'

In [ ]:
# Files get loaded from fairCorrect github repository
url = 'https://raw.githubusercontent.com/vladoxNCL/fairCorrect/master/Datasets/'
# Get balancing feature to end column
df = pd.read_csv(url + 'adult.data', header=None)

m, n= df.shape
print("IN - Rows: ", m, " Features: ",n)

# Assign names to columns
names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
         'occupation', 'relationship', 'race', 'sex', 'capital-gain',
         'capital-loss', 'hours-per-week', 'native-country', 'label']

df.columns = names

# Select firt 3 rows 
#df = df[:3]
#df = df[['workclass', 'fnlwgt', 'race', 'sex','label']]

# Create a new provenance document 
p = pr.Provenance(df, savepath + 'census_prov')
#p = pr_lib.Provenance(df, savepath)

IN - Rows:  32561  Features:  15


In [ ]:
#OPERATION 0
# Cleanup names from spaces
col = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'label']
#col = ['workclass', 'sex', 'label']

for c in col:
    df[c] = df[c].map(str.strip)

#PROVENANCE 0
d = p.get_prov_feature_transformation(df, col, 'Cleanup names from spaces')


In [ ]:
#OPERATION 1
# Replace ? character for NaN value
df = df.replace('?', np.nan)

#PROVENANCE 1
d = p.get_prov_value_transformation(df,'?')

In [ ]:
#OPERATION 2
# One-hot encode categorical variables
col = ['workclass', 'race']

onehot_col_map = {}
for c in col:
    #TOADD: Get map column-val
    unique_val = df[c].unique().tolist()
    onehot_col_map[c] = unique_val
    
    dummies = []
    dummies.append(pd.get_dummies(df[c]))
    df_dummies = pd.concat(dummies, axis = 1)
    df = pd.concat((df, df_dummies), axis = 1)
    df = df.drop([c], axis = 1)

#PROVENANCE 2
d = p.get_prov_onehot_encode(df, col, onehot_col_map)

In [ ]:
#OPERATION 3
# Assign sex and label binary values 0 and 1
df.sex = df.sex.replace('Male', 1)
df.sex = df.sex.replace('Female', 0)
df.label = df.label.replace('<=50K', 0)
df.label = df.label.replace('>50K', 1)

#PROVENANCE 3
col = ['sex', 'label']
d = p.get_prov_feature_transformation(df, col, 'Assign sex and label binary values 0 and 1')

In [ ]:
#OPERATION 4
# Drop fnlwgt variable
df = df.drop(['fnlwgt'], axis=1)
#PROVENANCE 4
d = p.get_prov_dim_reduction(df)

m, n= df.shape
print("OUT- Rows: ", m, " Features: ",n)

# Uncomment bottom line to save clean csv
#df.to_csv(savepath + 'income_onehot.csv', index=False)

#namefile = savepath + 'census_prov'
#p.save_all_graph(namefile)
#Image(namefile + '.png')
